# Evaluate RS

In [1]:
import psycopg2
import pandas as pd
import requests

In [2]:
def connect_and_query(query: str, params):
    conn = psycopg2.connect(
        host="localhost",
        port=5432,
        database="mp_dump",
        user="postgres",
        password="changeme"
    )

    cur = conn.cursor()

    cur.execute(query, params)
    res = cur.fetchall()

    cur.close()

    return res

In [3]:
def request_recommendations(viewed_service, purchase_history, metadata_weight, view_weight):
    data = {
        "service_id": viewed_service,
        "purchase_ids": purchase_history,
        "num": 5,
        "view_weight": view_weight,
        "metadata_weight": metadata_weight 
    }

    r = requests.post('http://0.0.0.0:4559/rs_evaluation/recommendation', json=data)
    
    return r.json()['service_ids']

In [4]:
EOSC_SERVICES_URL = "https://marketplace.eosc-portal.eu/services/"

def get_service_info(service_id):
    query = f"""
        SELECT id, name, slug
        FROM services
        WHERE id={service_id}
    """
    
    res = pd.DataFrame(connect_and_query(query, ()), columns=["id", "name", "slug"])
    res['slug'] = res['slug'].apply(lambda x: EOSC_SERVICES_URL + x)
    
    return res.to_dict()

In [5]:
def present_recommendations(viewed_service, purchase_history, metadata_weight=0.5, view_weight=0.5):
    viewed_df = get_service_info(viewed_service)
    purchase_history_df = [get_service_info(service_id) for service_id in purchase_history]
    
    recommended_ids = request_recommendations(viewed_service, purchase_history, metadata_weight, view_weight)
    recommended_df = [get_service_info(service_id) for service_id in recommended_ids]
    
    
    print("#"*180)
    print("> Currently viewing:")
    print(f"\t- {viewed_df['id'][0]} | {viewed_df['name'][0]} | {viewed_df['slug'][0]}")
    print()
    
    print("> Purchase history:")
    for service in purchase_history_df:
        print(f"\t- {service['id'][0]} | {service['name'][0]} | {service['slug'][0]}")
    print()
    
    print("> Recommendations:")
    for service in recommended_df:
        print(f"\t- {service['id'][0]} | {service['name'][0]} | {service['slug'][0]}")

## Experiments

**100 Percent IT Cloud**

In [6]:
present_recommendations(76, [], metadata_weight=0.5, view_weight=0.5)

####################################################################################################################################################################################
> Currently viewing:
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Purchase history:

> Recommendations:
	- 55 | ePouta Virtual Private Cloud | https://marketplace.eosc-portal.eu/services/csc-epouta
	- 177 | European Cloud Hosting | https://marketplace.eosc-portal.eu/services/european-cloud-hosting
	- 2 | EGI High-Throughput Compute | https://marketplace.eosc-portal.eu/services/egi-high-throughput-compute
	- 1 | EGI Cloud Compute | https://marketplace.eosc-portal.eu/services/egi-cloud-compute
	- 69 | Embassy Cloud | https://marketplace.eosc-portal.eu/services/embassy-cloud


In [7]:
present_recommendations(76, [], metadata_weight=1)

# We can see services that do not relate with the service we currently view EGI Check-In, Open Knowledge Maps

####################################################################################################################################################################################
> Currently viewing:
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Purchase history:

> Recommendations:
	- 55 | ePouta Virtual Private Cloud | https://marketplace.eosc-portal.eu/services/csc-epouta
	- 2 | EGI High-Throughput Compute | https://marketplace.eosc-portal.eu/services/egi-high-throughput-compute
	- 111 | Rahti Container Cloud | https://marketplace.eosc-portal.eu/services/rahti-container-cloud
	- 45 | EGI Check-In | https://marketplace.eosc-portal.eu/services/egi-check-in
	- 362 | Open Knowledge Maps | https://marketplace.eosc-portal.eu/services/open-knowledge-maps


In [8]:
present_recommendations(76, [], metadata_weight=0)

# Similar to metada_weight=0.5

####################################################################################################################################################################################
> Currently viewing:
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Purchase history:

> Recommendations:
	- 511 | CLOUDIFIN | https://marketplace.eosc-portal.eu/services/cloudifin
	- 1 | EGI Cloud Compute | https://marketplace.eosc-portal.eu/services/egi-cloud-compute
	- 79 | MetaCentrum Cloud | https://marketplace.eosc-portal.eu/services/metacentrum-cloud
	- 227 | Open Telekom Cloud | https://marketplace.eosc-portal.eu/services/open-telekom-cloud
	- 622 | SCIGNE Cloud Compute | https://marketplace.eosc-portal.eu/services/scigne-cloud-compute


In [9]:
present_recommendations(76, [419], metadata_weight=0.5)

# Interesting recommendation: de.NBI Cloud: Cloud Computing for Life Sciences which combines both viewing and history

####################################################################################################################################################################################
> Currently viewing:
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Purchase history:
	- 419 | DisVis web portal | https://marketplace.eosc-portal.eu/services/disvis-web-portal-6eab178c-9bc5-4c62-b7ce-aeeb18d5cba9

> Recommendations:
	- 69 | Embassy Cloud | https://marketplace.eosc-portal.eu/services/embassy-cloud
	- 177 | European Cloud Hosting | https://marketplace.eosc-portal.eu/services/european-cloud-hosting
	- 363 | de.NBI Cloud: Cloud Computing for Life Sciences | https://marketplace.eosc-portal.eu/services/cloud-computing-for-life-sciences
	- 425 | PDB-Tools web | https://marketplace.eosc-portal.eu/services/pdb-tools-web
	- 42 | CloudFerro Infrastructure | https://marketplace.eosc-portal.eu/services/cloudferro-infrastructure


In [10]:
present_recommendations(76, [419], metadata_weight=1)

# Focuses on web portals and offers 1 cloud solution

####################################################################################################################################################################################
> Currently viewing:
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Purchase history:
	- 419 | DisVis web portal | https://marketplace.eosc-portal.eu/services/disvis-web-portal-6eab178c-9bc5-4c62-b7ce-aeeb18d5cba9

> Recommendations:
	- 420 | HADDOCK2.4 web portal | https://marketplace.eosc-portal.eu/services/haddock2-4-web-portal
	- 421 | PowerFit web portal | https://marketplace.eosc-portal.eu/services/powerfit-web-portal-b8ddee6c-78f5-43d8-a5a2-9e3b7f1cb24e
	- 425 | PDB-Tools web | https://marketplace.eosc-portal.eu/services/pdb-tools-web
	- 422 | SpotOn web portal | https://marketplace.eosc-portal.eu/services/spoton-c5db8fd5-a546-4342-8bae-2b2b4777b67e
	- 69 | Embassy Cloud | https://marketplace.eosc-portal.eu/services/embassy-cloud


In [11]:
present_recommendations(76, [419], metadata_weight=0.5, view_weight=0.8)

####################################################################################################################################################################################
> Currently viewing:
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Purchase history:
	- 419 | DisVis web portal | https://marketplace.eosc-portal.eu/services/disvis-web-portal-6eab178c-9bc5-4c62-b7ce-aeeb18d5cba9

> Recommendations:
	- 177 | European Cloud Hosting | https://marketplace.eosc-portal.eu/services/european-cloud-hosting
	- 69 | Embassy Cloud | https://marketplace.eosc-portal.eu/services/embassy-cloud
	- 55 | ePouta Virtual Private Cloud | https://marketplace.eosc-portal.eu/services/csc-epouta
	- 2 | EGI High-Throughput Compute | https://marketplace.eosc-portal.eu/services/egi-high-throughput-compute
	- 1 | EGI Cloud Compute | https://marketplace.eosc-portal.eu/services/egi-cloud-compute


In [12]:
present_recommendations(76, [419, 352], metadata_weight=0.5, view_weight=0.5)

####################################################################################################################################################################################
> Currently viewing:
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Purchase history:
	- 419 | DisVis web portal | https://marketplace.eosc-portal.eu/services/disvis-web-portal-6eab178c-9bc5-4c62-b7ce-aeeb18d5cba9
	- 352 | E-Learning Platform of GBIF Spain | https://marketplace.eosc-portal.eu/services/e-learning-platform-of-gbif-spain

> Recommendations:
	- 69 | Embassy Cloud | https://marketplace.eosc-portal.eu/services/embassy-cloud
	- 177 | European Cloud Hosting | https://marketplace.eosc-portal.eu/services/european-cloud-hosting
	- 363 | de.NBI Cloud: Cloud Computing for Life Sciences | https://marketplace.eosc-portal.eu/services/cloud-computing-for-life-sciences
	- 2 | EGI High-Throughput Compute | https://marketplace.eosc-portal.eu/se

In [13]:
present_recommendations(352, [419, 76], metadata_weight=0.5, view_weight=0.5)

####################################################################################################################################################################################
> Currently viewing:
	- 352 | E-Learning Platform of GBIF Spain | https://marketplace.eosc-portal.eu/services/e-learning-platform-of-gbif-spain

> Purchase history:
	- 419 | DisVis web portal | https://marketplace.eosc-portal.eu/services/disvis-web-portal-6eab178c-9bc5-4c62-b7ce-aeeb18d5cba9
	- 76 | 100 Percent IT Trusted Cloud | https://marketplace.eosc-portal.eu/services/100-percent-it-trusted-cloud

> Recommendations:
	- 363 | de.NBI Cloud: Cloud Computing for Life Sciences | https://marketplace.eosc-portal.eu/services/cloud-computing-for-life-sciences
	- 210 | Software Integration Support | https://marketplace.eosc-portal.eu/services/software-integration-support
	- 51 | PaN faas | https://marketplace.eosc-portal.eu/services/pan-faas
	- 511 | CLOUDIFIN | https://marketplace.eosc-portal.eu/services/cloudif

**KER - Keyword Extractor**

In [14]:
present_recommendations(496, [], metadata_weight=0.5, view_weight=0.5)
# Proposes similar solutions from the same team

####################################################################################################################################################################################
> Currently viewing:
	- 496 | KER - Keyword Extractor | https://marketplace.eosc-portal.eu/services/ker-keyword-extractor

> Purchase history:

> Recommendations:
	- 493 | MorphoDiTa | https://marketplace.eosc-portal.eu/services/morphodita
	- 495 | Machine Translation | https://marketplace.eosc-portal.eu/services/machine-translation
	- 492 | ElixirFM | https://marketplace.eosc-portal.eu/services/elixirfm
	- 494 | NameTag | https://marketplace.eosc-portal.eu/services/nametag
	- 434 | UDPipe: tool for lemmatization, morphological analysis, POS tagging and dependency parsing in multiple languages | https://marketplace.eosc-portal.eu/services/udpipe-tool-for-lemmatization-morphological-analysis-pos-tagging-and-dependency-parsing-in-multiple-languages


In [15]:
present_recommendations(496, [], metadata_weight=0, view_weight=0.5)
# While the first two are related (with the first one being from a diff organisation) the rest of them are not

####################################################################################################################################################################################
> Currently viewing:
	- 496 | KER - Keyword Extractor | https://marketplace.eosc-portal.eu/services/ker-keyword-extractor

> Purchase history:

> Recommendations:
	- 626 | The Tromsø Repository of Language and Linguistics (TROLLing) | https://marketplace.eosc-portal.eu/services/the-tromso-repository-of-language-and-linguistics-trolling
	- 434 | UDPipe: tool for lemmatization, morphological analysis, POS tagging and dependency parsing in multiple languages | https://marketplace.eosc-portal.eu/services/udpipe-tool-for-lemmatization-morphological-analysis-pos-tagging-and-dependency-parsing-in-multiple-languages
	- 145 | V-SEEM CLOWDER | https://marketplace.eosc-portal.eu/services/v-seem-clowder
	- 424 | AMBER-based Portal Server for NMR structures (AMPS-NMR) | https://marketplace.eosc-portal.eu/services/amber-b

In [16]:
present_recommendations(496, [17], metadata_weight=0.5, view_weight=0.5)
# Virtual Language Observatory relates with the purchase history

####################################################################################################################################################################################
> Currently viewing:
	- 496 | KER - Keyword Extractor | https://marketplace.eosc-portal.eu/services/ker-keyword-extractor

> Purchase history:
	- 17 | Language Resource Switchboard | https://marketplace.eosc-portal.eu/services/language-resource-switchboard

> Recommendations:
	- 495 | Machine Translation | https://marketplace.eosc-portal.eu/services/machine-translation
	- 493 | MorphoDiTa | https://marketplace.eosc-portal.eu/services/morphodita
	- 18 | Virtual Collection Registry | https://marketplace.eosc-portal.eu/services/virtual-collection-registry
	- 492 | ElixirFM | https://marketplace.eosc-portal.eu/services/elixirfm
	- 16 | Virtual Language Observatory | https://marketplace.eosc-portal.eu/services/virtual-language-observatory


**Evaluate with use cases**

1) **User description**: Graduate of the History and Archeology department currently studying for her Msc in    linguistics

   **Use case**: Using the platform to search tools related to enity recognision in natural language texts.
   

In [17]:
present_recommendations(494, [])

####################################################################################################################################################################################
> Currently viewing:
	- 494 | NameTag | https://marketplace.eosc-portal.eu/services/nametag

> Purchase history:

> Recommendations:
	- 493 | MorphoDiTa | https://marketplace.eosc-portal.eu/services/morphodita
	- 492 | ElixirFM | https://marketplace.eosc-portal.eu/services/elixirfm
	- 495 | Machine Translation | https://marketplace.eosc-portal.eu/services/machine-translation
	- 496 | KER - Keyword Extractor | https://marketplace.eosc-portal.eu/services/ker-keyword-extractor
	- 626 | The Tromsø Repository of Language and Linguistics (TROLLing) | https://marketplace.eosc-portal.eu/services/the-tromso-repository-of-language-and-linguistics-trolling


In [18]:
present_recommendations(494, [137, 101])

####################################################################################################################################################################################
> Currently viewing:
	- 494 | NameTag | https://marketplace.eosc-portal.eu/services/nametag

> Purchase history:
	- 137 | Digital Humanities and Cultural Heritage OpenAIRE Community Gateway | https://marketplace.eosc-portal.eu/services/digital-humanities-and-cultural-heritage-openaire-community-gateway
	- 101 | Europeana APIs | https://marketplace.eosc-portal.eu/services/europeana-apis

> Recommendations:
	- 493 | MorphoDiTa | https://marketplace.eosc-portal.eu/services/morphodita
	- 495 | Machine Translation | https://marketplace.eosc-portal.eu/services/machine-translation
	- 492 | ElixirFM | https://marketplace.eosc-portal.eu/services/elixirfm
	- 496 | KER - Keyword Extractor | https://marketplace.eosc-portal.eu/services/ker-keyword-extractor
	- 142 | BiOnym | https://marketplace.eosc-portal.eu/services/bi

2) **User description**: A full-stack developer, member of a startup related to bioinformatics

   **Use case**: He is searching for anonymization tool for their collected datasets

In [19]:
present_recommendations(11, [])

####################################################################################################################################################################################
> Currently viewing:
	- 11 | B2ACCESS | https://marketplace.eosc-portal.eu/services/b2access

> Purchase history:

> Recommendations:
	- 6 | B2HANDLE | https://marketplace.eosc-portal.eu/services/b2handle
	- 9 | B2SHARE | https://marketplace.eosc-portal.eu/services/b2share
	- 8 | B2SAFE | https://marketplace.eosc-portal.eu/services/b2safe
	- 45 | EGI Check-In | https://marketplace.eosc-portal.eu/services/egi-check-in
	- 515 | OpenAIRE Login | https://marketplace.eosc-portal.eu/services/openaire-login


In [20]:
present_recommendations(11, [429, 419])

####################################################################################################################################################################################
> Currently viewing:
	- 11 | B2ACCESS | https://marketplace.eosc-portal.eu/services/b2access

> Purchase history:
	- 429 | MetalPDB: a database of metal-binding sites in 3D structures of biological macromolecules | https://marketplace.eosc-portal.eu/services/metalpdb-21ea9621-ff08-4e08-8c38-366d7aa07c88
	- 419 | DisVis web portal | https://marketplace.eosc-portal.eu/services/disvis-web-portal-6eab178c-9bc5-4c62-b7ce-aeeb18d5cba9

> Recommendations:
	- 420 | HADDOCK2.4 web portal | https://marketplace.eosc-portal.eu/services/haddock2-4-web-portal
	- 422 | SpotOn web portal | https://marketplace.eosc-portal.eu/services/spoton-c5db8fd5-a546-4342-8bae-2b2b4777b67e
	- 9 | B2SHARE | https://marketplace.eosc-portal.eu/services/b2share
	- 6 | B2HANDLE | https://marketplace.eosc-portal.eu/services/b2handle
	- 425 | P

In [21]:
present_recommendations(11, [429, 419, 247, 109])

####################################################################################################################################################################################
> Currently viewing:
	- 11 | B2ACCESS | https://marketplace.eosc-portal.eu/services/b2access

> Purchase history:
	- 429 | MetalPDB: a database of metal-binding sites in 3D structures of biological macromolecules | https://marketplace.eosc-portal.eu/services/metalpdb-21ea9621-ff08-4e08-8c38-366d7aa07c88
	- 419 | DisVis web portal | https://marketplace.eosc-portal.eu/services/disvis-web-portal-6eab178c-9bc5-4c62-b7ce-aeeb18d5cba9
	- 247 | LEMONADE - Live Exploration and Mining Of a Non-trivial Amount of Data from Everywhere | https://marketplace.eosc-portal.eu/services/lemonade-live-exploration-and-mining-of-a-non-trivial-amount-of-data-from-everywhere
	- 109 | NOMAD repository | https://marketplace.eosc-portal.eu/services/nomad-repository

> Recommendations:
	- 9 | B2SHARE | https://marketplace.eosc-portal.e

3) **User description**: A practising phycologist

   **Use case**: Search for a registry with surveys about migrants and refugees

In [22]:
present_recommendations(416, [])

####################################################################################################################################################################################
> Currently viewing:
	- 416 | ISIDORE | https://marketplace.eosc-portal.eu/services/isidore

> Purchase history:

> Recommendations:
	- 17 | Language Resource Switchboard | https://marketplace.eosc-portal.eu/services/language-resource-switchboard
	- 92 | OpenAIRE Mining Service | https://marketplace.eosc-portal.eu/services/openaire-mining-service
	- 106 | Data Service Portal Aila | https://marketplace.eosc-portal.eu/services/data-service-portal-aila
	- 137 | Digital Humanities and Cultural Heritage OpenAIRE Community Gateway | https://marketplace.eosc-portal.eu/services/digital-humanities-and-cultural-heritage-openaire-community-gateway
	- 660 | OpenBioMaps | https://marketplace.eosc-portal.eu/services/openbiomaps


In [23]:
present_recommendations(416, [408, 668])

####################################################################################################################################################################################
> Currently viewing:
	- 416 | ISIDORE | https://marketplace.eosc-portal.eu/services/isidore

> Purchase history:
	- 408 | ELSST – European Language Social Science Thesaurus | https://marketplace.eosc-portal.eu/services/elsst-european-language-social-science-thesaurus
	- 668 | European Social Survey (ESS) as a service | https://marketplace.eosc-portal.eu/services/european-social-survey-ess-as-a-service

> Recommendations:
	- 106 | Data Service Portal Aila | https://marketplace.eosc-portal.eu/services/data-service-portal-aila
	- 17 | Language Resource Switchboard | https://marketplace.eosc-portal.eu/services/language-resource-switchboard
	- 92 | OpenAIRE Mining Service | https://marketplace.eosc-portal.eu/services/openaire-mining-service
	- 387 | OPERAS Research for Society (Hypotheses) | https://marketplace.e

4) **User description**: An employee at the national observatory of Athens

   **Use case**: Searching for a tool for terrain mapping

In [24]:
present_recommendations(521, [])

####################################################################################################################################################################################
> Currently viewing:
	- 521 | UW-Mos | https://marketplace.eosc-portal.eu/services/uw-mos

> Purchase history:

> Recommendations:
	- 560 | VD-Maps | https://marketplace.eosc-portal.eu/services/vd-maps
	- 507 | UW-MAP | https://marketplace.eosc-portal.eu/services/uw-map
	- 554 | ADAM Platform | https://marketplace.eosc-portal.eu/services/adam-platform
	- 661 | LOFAR Science Processing | https://marketplace.eosc-portal.eu/services/lofar-science-processing
	- 663 | Climadjust | https://marketplace.eosc-portal.eu/services/climadjust


In [25]:
present_recommendations(521, [37, 15, 26])

####################################################################################################################################################################################
> Currently viewing:
	- 521 | UW-Mos | https://marketplace.eosc-portal.eu/services/uw-mos

> Purchase history:
	- 37 | EODC Data Catalogue Service | https://marketplace.eosc-portal.eu/services/eodc-data-catalogue-service
	- 15 | GEO Discovery and Access Broker | https://marketplace.eosc-portal.eu/services/geo-dab
	- 26 | OPENCoastS Portal | https://marketplace.eosc-portal.eu/services/opencoasts-portal

> Recommendations:
	- 507 | UW-MAP | https://marketplace.eosc-portal.eu/services/uw-map
	- 560 | VD-Maps | https://marketplace.eosc-portal.eu/services/vd-maps
	- 178 | Indian Ocean Tuna Commission Spatial Data Catalog | https://marketplace.eosc-portal.eu/services/indian-ocean-tuna-commission-spatial-data-catalog
	- 225 | Global Tuna Atlas Spatial Data Catalog | https://marketplace.eosc-portal.eu/services/globa

5) **User description**: Phd vet

   **Use case**: General search for new tools related to his profession

In [26]:
present_recommendations(348, [])

####################################################################################################################################################################################
> Currently viewing:
	- 348 | Ubora | https://marketplace.eosc-portal.eu/services/ubora

> Purchase history:

> Recommendations:
	- 184 | UBORA e-platform | https://marketplace.eosc-portal.eu/services/ubora-e-platform
	- 174 | Scientific Training Environment | https://marketplace.eosc-portal.eu/services/scientific-training-environment
	- 159 | OpenMinTeD Support and Training | https://marketplace.eosc-portal.eu/services/openminted-support-and-training
	- 102 | Snap4City | https://marketplace.eosc-portal.eu/services/snap4city
	- 210 | Software Integration Support | https://marketplace.eosc-portal.eu/services/software-integration-support


In [27]:
present_recommendations(348, [514, 138])

####################################################################################################################################################################################
> Currently viewing:
	- 348 | Ubora | https://marketplace.eosc-portal.eu/services/ubora

> Purchase history:
	- 514 | GBIF Species Occurrence Data | https://marketplace.eosc-portal.eu/services/gbif-species-occurrence-data
	- 138 | PhenoMeNal | https://marketplace.eosc-portal.eu/services/phenomenal

> Recommendations:
	- 184 | UBORA e-platform | https://marketplace.eosc-portal.eu/services/ubora-e-platform
	- 174 | Scientific Training Environment | https://marketplace.eosc-portal.eu/services/scientific-training-environment
	- 352 | E-Learning Platform of GBIF Spain | https://marketplace.eosc-portal.eu/services/e-learning-platform-of-gbif-spain
	- 156 | D4Science Spatial Data Catalog | https://marketplace.eosc-portal.eu/services/d4science-spatial-data-catalog
	- 40 | Datacube | https://marketplace.eosc-portal.e